In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,48728
2,Huelva,Confirmados PDIA 14 días,1421
3,Huelva,Tasa PDIA 14 días,"276,9062883644796"
4,Huelva,Confirmados PDIA 7 días,785
5,Huelva,Tasa PDIA 7 dias,"152,97075043357952"
6,Huelva,Total Confirmados,48936
7,Huelva,Curados,46008
8,Huelva,Fallecidos,424


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  48728.0


/tmp/ipykernel_17706/1396922249.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  13582.0


/tmp/ipykernel_17706/2509482988.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

/tmp/ipykernel_17706/2690025379.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
/tmp/ipykernel_17706/2690025379.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
/tmp/ipykernel_17706/2690025379.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 560 personas en los últimos 7 días 

Un positivo PCR cada 297 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,48728.0,785.0,1421.0,513170.0,152.970750,276.906288,242.0
Huelva-Costa,28756.0,432.0,787.0,289548.0,149.198060,271.802948,133.0
Huelva (capital),13582.0,257.0,485.0,143837.0,178.674472,337.187233,78.0
Condado-Campiña,15190.0,228.0,444.0,156231.0,145.937746,284.194558,69.0
Sierra de Huelva-Andévalo Central,4351.0,122.0,187.0,67391.0,181.033076,277.485124,39.0
Almonte,2438.0,51.0,68.0,24507.0,208.103807,277.471743,23.0
Aljaraque,1758.0,44.0,77.0,21474.0,204.898948,358.573158,14.0
Aracena,726.0,44.0,49.0,8255.0,533.010297,593.579649,13.0
Punta Umbría,1362.0,37.0,75.0,15355.0,240.963855,488.440247,13.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Puerto Moral,14.0,9.0,9.0,278.0,3237.410072,3237.410072,4.0
Granada de Río-Tinto (La),18.0,1.0,4.0,243.0,411.522634,1646.090535,0.0
Paterna del Campo,280.0,7.0,48.0,3457.0,202.487706,1388.487128,0.0
Cumbres Mayores,157.0,19.0,21.0,1749.0,1086.335049,1200.686106,8.0
Corteconcepción,51.0,2.0,5.0,536.0,373.134328,932.835821,1.0
Aracena,726.0,44.0,49.0,8255.0,533.010297,593.579649,13.0
Villanueva de las Cruces,23.0,2.0,2.0,387.0,516.795866,516.795866,NaN
Bollullos Par del Condado,1421.0,35.0,73.0,14387.0,243.275179,507.402516,9.0
Punta Umbría,1362.0,37.0,75.0,15355.0,240.963855,488.440247,13.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Manzanilla,153.0,1.0,8.0,2118.0,47.214353,377.714825,0.0,0.125000
Paterna del Campo,280.0,7.0,48.0,3457.0,202.487706,1388.487128,0.0,0.145833
Higuera de la Sierra,75.0,1.0,4.0,1291.0,77.459334,309.837335,0.0,0.250000
Granada de Río-Tinto (La),18.0,1.0,4.0,243.0,411.522634,1646.090535,0.0,0.250000
Zalamea la Real,130.0,1.0,4.0,3054.0,32.743942,130.975769,0.0,0.250000
Escacena del Campo,242.0,2.0,7.0,2287.0,87.450809,306.077831,2.0,0.285714
Rociana del Condado,701.0,8.0,22.0,7939.0,100.768359,277.112987,2.0,0.363636
San Juan del Puerto,782.0,4.0,10.0,9411.0,42.503453,106.258634,1.0,0.400000
Corteconcepción,51.0,2.0,5.0,536.0,373.134328,932.835821,1.0,0.400000
